In [ ]:
# Differential DSP: https://github.com/magenta/ddsp
# DDSP makes traditional DSP components (e.g., oscillators, filters, reverbs) differentiable, 
# allowing them to be trained using gradient-based optimization.
# DDSP models are more interpretable than black-box neural networks because they use components 
# based on physical and mathematical principles (e.g., harmonic synthesis, spectral modeling).
# DDSP excels in tasks like: pitch shifting, timbre transfer, audio synthesis, audio effects

import ddsp
import ddsp.training
import numpy as np
import tensorflow as tf
import librosa
import soundfile as sf

audio_path = './Data/5sec_440Hz.wav'  

# Resample to 16kHz
audio, sr = librosa.load(audio_path, sr=16000) 

# Convert audio to a tensor
audio_tensor = tf.convert_to_tensor(audio, dtype=tf.float32)

# Extract fundamental frequency (f0) and loudness
f0_hz, loudness_db = ddsp.training.metrics.compute_f0_loudness(audio_tensor, sr)

# Apply pitch shifting (e.g., shift up by 2 semitones)
# A semitone is the smallest musical interval commonly used in Western music. 
# It represents the pitch difference between two adjacent notes in a 12-tone equal temperament scale 
semitone_shift = 2
f0_hz_shifted = f0_hz * (2 ** (semitone_shift / 12.0))

# Synthesize the audio with the shifted pitch
harmonic = ddsp.synths.Harmonic()
synthesized_audio = harmonic(f0_hz=f0_hz_shifted, amplitudes=tf.ones_like(f0_hz))

# Convert the synthesized audio to a NumPy array
synthesized_audio_np = synthesized_audio.numpy()

output_path = './Output/ddsp_output_pitch_shifted.wav'
sf.write(output_path, synthesized_audio_np, sr)
